In [1]:
!pip install -q kaggle

from google.colab import files

files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"muhammad4hmed","key":"a317a4fe658a1328ab0a5549b9296b17"}'}

In [5]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d musha2017/shopee-external-models

100% 1.26G/1.26G [00:19<00:00, 69.0MB/s]



In [7]:
!unzip /content/shopee-external-models.zip

Archive:  /content/shopee-external-models.zip
  inflating: EfficientNetB3_M0.7_512_42.h5  
  inflating: Keras_Applications-1.0.8-py3-none-any.whl  
  inflating: bert_en_uncased_L-24_H-1024_A-16_1/assets/vocab.txt  
  inflating: bert_en_uncased_L-24_H-1024_A-16_1/saved_model.pb  
  inflating: bert_en_uncased_L-24_H-1024_A-16_1/variables/variables.data-00000-of-00001  
  inflating: bert_en_uncased_L-24_H-1024_A-16_1/variables/variables.index  
  inflating: efficientnet-1.1.0-py3-none-any.whl  
  inflating: tokenization.py         


In [8]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 13.6MB/s 


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this notebook, we are going to train a model to extract word embeddings so that we can use them in future and we can have more better context for words.

In [10]:
from shutil import copyfile
import re
import os
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
copyfile(src = "/content/tokenization.py", dst = "okenization.py")
import tokenization
import tensorflow_hub as hub

In [16]:
# Configuration
EPOCHS = 10
BATCH_SIZE = 32
# Seed
SEED = 123
# Verbosity
VERBOSE = 1
LR = 0.00001

In [23]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def read_and_preprocess():
    df = pd.read_csv('/content/drive/MyDrive/Clean_Dataset.csv')
    # tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
    # df['matches'] = df['label_group'].map(tmp)
    # df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
    encoder = LabelEncoder()
    df['up_votes'] = encoder.fit_transform(df['up_votes'])
    N_CLASSES = df['up_votes'].nunique()
    print(f'We have {N_CLASSES} classes')
    x_train, x_val, y_train, y_val = train_test_split(df[['title']], df['up_votes'], shuffle = True, random_state = SEED, test_size = 0.33)
    return df, N_CLASSES, x_train, x_val, y_train, y_val

# Return tokens, masks and segments from a text array or series
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

# Function to build bert model
def build_bert_model(bert_layer, max_len = 512):
    
    margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )
    
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    label = tf.keras.layers.Input(shape = (), name = 'label')

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    x = margin([clf_output, label])
    output = tf.keras.layers.Softmax(dtype='float32')(x)
    model = tf.keras.models.Model(inputs = [input_word_ids, input_mask, segment_ids, label], outputs = [output])
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = LR),
                  loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

def load_train_and_evaluate(x_train, x_val, y_train, y_val):
    seed_everything(SEED)
    # Load BERT from the Tensorflow Hub
    module_url = "/content/bert_en_uncased_L-24_H-1024_A-16_1"
    bert_layer = hub.KerasLayer(module_url, trainable = True)
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
    x_train = bert_encode(x_train['title'].values, tokenizer, max_len = 12)
    x_val = bert_encode(x_val['title'].values, tokenizer, max_len = 12)
    y_train = y_train.values
    y_val = y_val.values
    # Add targets to train and val
    x_train = (x_train[0], x_train[1], x_train[2], y_train)
    x_val = (x_val[0], x_val[1], x_val[2], y_val)
    bert_model = build_bert_model(bert_layer, max_len = 12)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'Bert_{SEED}.h5', 
                                                    monitor = 'val_loss', 
                                                    verbose = VERBOSE, 
                                                    save_best_only = True,
                                                    save_weights_only = True, 
                                                    mode = 'min')
    history = bert_model.fit(x_train, y_train,
                             validation_data = (x_val, y_val),
                             epochs = EPOCHS, 
                             callbacks = [checkpoint],
                             batch_size = BATCH_SIZE,
                             verbose = VERBOSE)
    

In [24]:
df, N_CLASSES, x_train, x_val, y_train, y_val = read_and_preprocess()

We have 5782 classes


In [25]:
x_train['title'].fillna('NA',inplace=True)
x_val['title'].fillna('NA',inplace=True)

In [ ]:
load_train_and_evaluate(x_train, x_val, y_train, y_val)

Epoch 1/10
10663/10663 [==============================] - 2252s 209ms/step - loss: 18.4557 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 16.6567 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 16.65665, saving model to Bert_123.h5
Epoch 2/10
10663/10663 [==============================] - 2217s 208ms/step - loss: 15.0240 - sparse_categorical_accuracy: 6.4703e-04 - val_loss: 10.5151 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 16.65665 to 10.51507, saving model to Bert_123.h5
Epoch 3/10
10663/10663 [==============================] - 2215s 208ms/step - loss: 10.4172 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 10.6193 - val_sparse_categorical_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 10.51507
Epoch 4/10
10663/10663 [==============================] - 2215s 208ms/step - loss: 10.4353 - sparse_categorical_accuracy: 1.0010e-04 - val_loss: 10.4761 - val_sparse_categorical_acc

The above trained model can be used to derive the embeddings of sentences